In [ ]:
video_path = ''
out_path = ''

In [ ]:
import cv2
import smtplib
from email.mime.text import MIMEText
import time
import psutil
import os
from ultralytics import YOLO
from dotenv import load_dotenv


# Start time
start_time = time.time()

# Get process ID
process = psutil.Process(os.getpid())

# Load YOLO model (Nano version for speed)
model = YOLO("yolov8n.pt", verbose=False)

# Define video path
cap = cv2.VideoCapture(video_path)

# Email configuration
load_dotenv(dotenv_path=r'C:\Users\haridas\Desktop\PU\HT\smart-watch\smart-watch\.env')
sender_email = os.getenv('SENDER_EMAIL')
receiver_email = os.getenv('RECEIVER_EMAIL')
smtp_server = "smtp.gmail.com"
smtp_port = 587
email_password = os.getenv("SENDER_PASS")  # Store password securely

# Cooldown period (in seconds) for email alerts
cooldown_period = 300  # in seconds
last_trespass_alert_time = 0

# Motion detection variables
has_motion = False
motion_at = time.time()
check_period = 5

# Background subtractor for motion detection
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

def send_email_alert(timestamp):
    subject = "Trespassing Alert"
    body = f"A human trespassing event was detected at [Camera_ID] on {timestamp}."

    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = receiver_email

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, email_password)
        server.sendmail(sender_email, [receiver_email], msg.as_string())
        server.quit()
        print(f"Alert email sent at {timestamp}.")
    except Exception as e:
        print("Failed to send email:", e)

# FPS control
fps = 3
i = 1

# video saving control
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter("output.mp4", fourcc, 30, 
                      (int(cap.get(3)), int(cap.get(4))))


while True:
    ret, frame = cap.read()
    if not ret:
        break

    if has_motion:
        if i % fps != 0:
            i += 1
            continue
        i = 1

        # YOLO human detection
        results = model.predict(frame, verbose=False)
        human_detected = False

        for result in results:
            for box in result.boxes:
                if int(box.cls.item()) == 0 and box.conf >= 0.5:  # Confidence threshold
                    human_detected = True
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    
        current_time = time.time()
        if human_detected and current_time - last_trespass_alert_time >= cooldown_period:
            last_trespass_alert_time = time.time()
            detection_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)  # Get timestamp in milliseconds
            detection_time_sec = detection_time_ms / 1000  # Convert to seconds
            detection_time_str = time.strftime("%H:%M:%S", time.gmtime(detection_time_sec))  # Convert to hh:mm:ss format
            print("Detected Trespassing")
            send_email_alert(detection_time_str)  # Pass timestamp to email function


        if time.time() - motion_at > check_period:
            has_motion = False
            motion_at = time.time()

    else:
        human_detected = False
        fgMask = backSub.apply(frame)
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
        contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if 1000 > cv2.contourArea(contour) > 500:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = h / float(w)
                if aspect_ratio > 1.2:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    has_motion = True
                    motion_at = time.time()
                    break

    cv2.imshow("Frame", frame)
    out.write(frame)  # Save video to output.mp4 file
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# End time
end_time = time.time()

# Get memory usage in MB
memory_used = process.memory_info().rss / (1024 * 1024)

print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"Peak Memory Usage: {memory_used:.2f} MB")
